In [20]:
import json
import pandas as pd
import MeCab

In [21]:
stories_json = []
with open('./output_stories_0123.json', 'r') as f:
    stories_json = stories_json + json.loads(f.read())

print(len(stories_json))
print(stories_json[1].keys())

839
dict_keys(['index', 'stories_index', 'title', 'broadcasting_date', 'abstracts_list'])


In [22]:
stories_df = pd.DataFrame(stories_json)
stories_df["abstract_joined"] = stories_df["abstracts_list"].str.join(" ")
stories_df.head()

,index,stories_index,title,broadcasting_date,abstracts_list,abstract_joined
0,0325,0,「映画ドラえもん のび太の新魔界大冒険～７人の魔法使い～」,2013年3月15日,[ 何をやってもうまくいかないのび太は、「魔法（まほう）が使えたら便利で楽しくなるにちがいな...,何をやってもうまくいかないのび太は、「魔法（まほう）が使えたら便利で楽しくなるにちがいない...
1,0326,0,「ばくはつコショウ」,2013年4月12日,[ ある朝、ねぼうしたのび太に泣きつかれたドラえもんは、『ばくはつコショウ』を取り出す。この...,ある朝、ねぼうしたのび太に泣きつかれたドラえもんは、『ばくはつコショウ』を取り出す。このコ...
2,0326,1,「何が何でもお花見を」,2013年4月12日,[ みんなから家族でお花見に行った話を聞き、うらやましくてしかたのないのび太。さっそくママに...,みんなから家族でお花見に行った話を聞き、うらやましくてしかたのないのび太。さっそくママにた...
3,0327,0,「やりクリしてハワイ旅行」,2013年4月26日,[ スネ夫からハワイ旅行をじまんされたのび太は、自分もハワイに行きたいとママにたのもうとする...,スネ夫からハワイ旅行をじまんされたのび太は、自分もハワイに行きたいとママにたのもうとするが...
4,0327,1,「しずかちゃんのはごろも」,2013年4月26日,[ 学校で「天女と羽衣（はごろも）」の劇（げき）をやることになったのび太としずか。劇の最後、...,学校で「天女と羽衣（はごろも）」の劇（げき）をやることになったのび太としずか。劇の最後、天...


In [23]:
t = MeCab.Tagger('-d /opt/homebrew/lib/mecab/dic/mecab-ipadic-neologd')
def mecab_tokenizer(text):
    parsed_lines = t.parse(text).split("\n")[:-2]
    surfaces = [l.split('\t')[0] for l in parsed_lines]
    features = [l.split('\t')[1] for l in parsed_lines]
    # 原型を取得
    bases = [f.split(',')[6] for f in features]
    # 品詞を取得
    pos = [f.split(',')[0] for f in features]

    # 各単語を原型に変換する
    token_list = [b if b != '*' else s for s, b in zip(surfaces, bases)]
    # 品詞を絞り込み
    target_pos = ["名詞"]
    token_list = [t for t, p in zip(token_list, pos) if p in target_pos]
    # アルファベットを小文字に統一
    token_list = [t.lower() for t in token_list]

    return [ text for text in filter(lambda x: not x.isascii(), token_list)]

wt = MeCab.Tagger('-Owakati')
def mecab_tokenizer_wakachi(text):
    return wt.parse(text).split(" ")

In [24]:
# ストーリーが取得できたものに絞る
stories_df = stories_df[stories_df['abstract_joined'].str.len() > 0] 
sentences = stories_df["abstract_joined"].apply(mecab_tokenizer_wakachi).apply(set).apply(list)
print(sentences)
print(sentences)

0      [日, とともに, 元, \r, ！, ほう, 族, どんどん, 何, 戻そ, や, 美夜子...
1      [朝, 泣きつか, 一生, さっそく, \r, ！, とっ, せい, の, ながら, 出, ...
2      [あやつり, さっそく, 宣言, \r, みる, ！, 向かう, お, あり, キャンセル,...
3      [日, さっそく, はじめ, \r, と, ！, みごと, や, スネ, の, 言い出せ, ...
4      [外し, さっそく, なっ, なら, 言い, \r, 空, ！, ほう, よろこび, 山菜,...
                             ...                        
834    [たち, 庭, モテ, ！, 相手, そう, 何, スネ, の, ながら, 夜, ）, うか...
835    [じゃ, なっ, 言い, 急, なかなか, おす, 言っ, ！, 強がる, お, あり, の...
836    [風呂敷, たち, 遊び, きり, ごっこ, はっ, ！, ど, お, や, チーム, 満点...
837    [取っ, なっ, 言い, おそう, 青ざめ, みる, 言っ, ！, そう, 連続, 今度, ...
838    [たち, さっそく, なっ, 動か, ロボット, 夕方, もどっ, ！, 雪だるま, すごす...
Name: abstract_joined, Length: 839, dtype: object
0      [日, とともに, 元, \r, ！, ほう, 族, どんどん, 何, 戻そ, や, 美夜子...
1      [朝, 泣きつか, 一生, さっそく, \r, ！, とっ, せい, の, ながら, 出, ...
2      [あやつり, さっそく, 宣言, \r, みる, ！, 向かう, お, あり, キャンセル,...
3      [日, さっそく, はじめ, \r, と, ！, みごと, や, スネ, の, 言い出せ, ...
4      [外し, さっそく, なっ, なら, 言い, \r, 空, ！, ほう, よろこび, 山菜,...
                             ...      

In [29]:
print(stories_df[stories_df["abstract_joined"].str.contains("凶悪")]["abstract_joined"])

100    \r\n　今日はドラえもんの誕生日。ドラえもんをおどろかせるようなものをプレゼントしたいと考...
Name: abstract_joined, dtype: object


In [26]:
keywords_dict = {}
for keyword_list in sentences.tolist():
    for keyword in keyword_list:
        if (keyword not in keywords_dict): keywords_dict[keyword] = 1
        else: keywords_dict[keyword] = keywords_dict[keyword] + 1

keywords_df = pd.DataFrame([{
    "key": key,
    "val": val
} for key,val in keywords_dict.items()])
keywords_df = keywords_df.sort_values("val", ascending=False)


In [27]:
pd.set_option('display.max_rows', 500)
keywords_df

,key,val
82,を,839
116,た,839
15,の,839
105,、,839
53,,839
...,...,...
6478,温め,1
2847,発進,1
2852,スイムドリンク,1
6474,のり,1


In [28]:
import json
keywords_df.to_csv('doraemon-word-frequencies.csv')